In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
main_path = '../overfitting/setTraining/select_noOhe/'
train = np.load(main_path+'train.npy')
test = np.load(main_path+'test.npy')
y_train = np.load(main_path+'y_train.npy')
y_test = np.load(main_path+'y_test.npy')
print('shapes: train = {}, y_train = {}, test = {}, y_teste = {}'.format(train.shape,y_train.shape,test.shape,y_test.shape))

shapes: train = (17389, 12), y_train = (17389, 1), test = (8566, 12), y_teste = (8566, 1)


In [3]:
# standardscaler x data
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(train)
X_testScaled = sc_x.fit_transform(test)

## MLP CLASSIFIER

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [5]:
y = np.ravel(y_train)
print(y.shape)

(17389,)


In [6]:
mlp_rs = MLPClassifier(max_iter=500)
parameter_space = {
    'hidden_layer_sizes': [(10,12,10),(12,),(10,12,10,10),(12,12,12),(10,10,10,10,10),(256,128,64,32)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'batch_size':[10,50], 'power_t':[0.5],
}
clf = GridSearchCV(mlp_rs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_trainScaled, y) # X is train samples and y is the corresponding labels

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_iter=500,
                                     momentum=0.9, n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_sta...
             param_grid={'activation': ['tanh', 'relu', 'logistic'],
                         'alpha': [0.0001, 0.05], 'batch_size': [10, 50],
                         'hidden_layer_sizes': [(10, 12, 10), (12,),
                                                (10, 12, 10, 10), (12, 12, 12),
                            

In [7]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 50, 'hidden_layer_sizes': (256, 128, 64, 32), 'learning_rate': 'adaptive', 'power_t': 0.5, 'solver': 'sgd'}


In [8]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.768 (+/-0.015) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 10), 'learning_rate': 'constant', 'power_t': 0.5, 'solver': 'sgd'}
0.778 (+/-0.026) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 10), 'learning_rate': 'constant', 'power_t': 0.5, 'solver': 'adam'}
0.734 (+/-0.022) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 10), 'learning_rate': 'constant', 'power_t': 0.5, 'solver': 'lbfgs'}
0.777 (+/-0.030) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 10), 'learning_rate': 'adaptive', 'power_t': 0.5, 'solver': 'sgd'}
0.762 (+/-0.023) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 10), 'learning_rate': 'adaptive', 'power_t': 0.5, 'solver': 'adam'}
0.735 (+/-0.023) for {'activation': 'tanh', 'alpha': 0.0001, 'batch_size': 10, 'hidden_layer_sizes': (10, 12, 1

In [9]:
y_teste = np.ravel(y_test)
y_true, y_pred = y_teste , clf.predict(X_testScaled)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

         0.0       0.85      0.83      0.84      1753
         1.0       0.84      0.78      0.81      1707
         2.0       0.80      0.84      0.82      1706
         3.0       0.90      0.92      0.91      1732
         4.0       0.94      0.95      0.95      1668

    accuracy                           0.87      8566
   macro avg       0.87      0.87      0.87      8566
weighted avg       0.87      0.87      0.86      8566

